# Alejandro Paredes, Parameter tunning of BERT

In [1]:
!pip install transformers datasets peft evaluate


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [2]:
model_checkpoint = 'distilbert-base-uncased'

#Define label maps
id2label = {0:"UNDEFINED" ,1:"LEFT",2:"RIGHT",3:"CENTER"}
label2id = {"undefined": 0, "left": 1, "right": 2, "center": 3}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=4, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
from datasets import load_dataset

from transformers import BertTokenizerFast

from torch.utils.data import DataLoader

dataset = load_dataset("csv", data_files="C:/Users/ALEJANDRO/Downloads/3813664/2017_1.csv") 
dataset

FileNotFoundError: Unable to find 'C:/Users/ALEJANDRO/Downloads/3813664/2017_1.csv'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix=True)

In [8]:
def tokenize_function(examples):
  text = examples["body"]

  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors = "np",
      truncation = True,
      max_length = 512
  )

  return tokenized_inputs

In [6]:
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

In [9]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/146718 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'date_publish', 'outlet', 'headline', 'lead', 'body', 'authors', 'domain', 'url', 'political_leaning', 'input_ids', 'attention_mask'],
        num_rows: 146718
    })
})

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)
  return {"accuracy": accuracy.compute(predictions=predictions
                                       , references=labels)}

In [11]:
text_list = ["It was good.", "Not a fan, don't recommended",
              "Better than the first one."]

print("Untrained model")
for text in text_list:
  inputs = tokenizer.encode(text, return_tensors="pt")
  logits = model(inputs).logits
  predictions = torch.argmax(logits)
  print(f'{text} - {id2label[predictions.tolist()]}')

Untrained model
It was good. - Positive
Not a fan, don't recommended - Negative
Better than the first one. - Negative


In [12]:
peft_config = LoraConfig(task_type='SEQ_CLS',
                         r = 4,
                         lora_alpha=32,
                         lora_dropout=0.01,
                         target_modules = ['q_lin'])

In [13]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 630,532 || all params: 67,587,080 || trainable%: 0.9329


In [14]:
lr = 1e-3
batch_size = 10
num_epochs = 5

training_args = TrainingArguments(
    output_dir=""+model_checkpoint+"lora-txt",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
)

c:\Users\ALEJANDRO\Documents\5. Programming\.virtualenvs\aplt_duke\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

In [16]:
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.4470183849334717, 'eval_accuracy': {'accuracy': 0.865}, 'eval_runtime': 20.3094, 'eval_samples_per_second': 49.238, 'eval_steps_per_second': 12.31, 'epoch': 1.0}
{'loss': 0.4357, 'grad_norm': 25.799226760864258, 'learning_rate': 0.0006, 'epoch': 2.0}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.625886082649231, 'eval_accuracy': {'accuracy': 0.857}, 'eval_runtime': 20.5779, 'eval_samples_per_second': 48.596, 'eval_steps_per_second': 12.149, 'epoch': 2.0}


...

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.6084117293357849, 'eval_accuracy': {'accuracy': 0.889}, 'eval_runtime': 21.0686, 'eval_samples_per_second': 47.464, 'eval_steps_per_second': 11.866, 'epoch': 4.0}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.6403375267982483, 'eval_accuracy': {'accuracy': 0.889}, 'eval_runtime': 21.087, 'eval_samples_per_second': 47.423, 'eval_steps_per_second': 11.856, 'epoch': 5.0}
{'train_runtime': 299.7413, 'train_samples_per_second': 16.681, 'train_steps_per_second': 4.17, 'train_loss': 0.24538958206176759, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.24538958206176759, metrics={'train_runtime': 299.7413, 'train_samples_per_second': 16.681, 'train_steps_per_second': 4.17, 'total_flos': 556790525519424.0, 'train_loss': 0.24538958206176759, 'epoch': 5.0})

In [17]:
model.to('cuda')
print('Trained model predictions')
for text in text_list:
  inputs = tokenizer.encode(text, return_tensors='pt').to('cuda')

  logits = model(inputs).logits
  predictions = torch.max(logits,1).indices

  print(f'{text} - {id2label[predictions.tolist()[0]]}')

Trained model predictions
It was good. - Positive
Not a fan, don't recommended - Negative
Better than the first one. - Positive


In [26]:
output_model_file = 'pytorch_distilbert_imbd.bin'
output_vocab_file = 'vocab_distilbert_imbd.bin'

# Save model
model_to_save = model
torch.save(model_to_save, output_model_file)

# Save tokenizer vocabulary in the current directory
tokenizer.save_vocabulary(".")  # Current directory

# Save model state dictionary
torch.save(model.state_dict(), 'trained_model_gral_imbd.pth')

print('All files saved')



All files saved
